# MBTI Prediction

In [23]:
import pandas as pd
import numpy as np

df = pd.read_csv("./data/mbti_1.csv", header = 0)

df.head()


,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


## Dataset Analysis

## Learning

In [48]:
data = df
print(data.shape)
# print(posts.iloc[6376])

(8675, 2)


### Split dataset in train and test sets

In [45]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.4, random_state=0)
print(train.shape, test.shape)
train.head()

(5205, 2) (3470, 2)


,type,posts
4233,ISTP,'boring|||http://www.youtube.com/watch?v=c1KNZ...
1273,INFP,'The Americans It's a new show and I really ...
3092,INTJ,No one under stands your sense of humor. 8653...
6376,ISTP,"'I admire you humor, INTP. So, it would be gre..."
7643,INFP,'I was wondering if anyone would take the time...


### Preprocessing

In [49]:
from preprocessing import format_text

train = pd.DataFrame(train)
train['preprocessed_posts'] = train['posts'].apply(format_text)
train.head()

,type,posts,preprocessed_posts
4233,ISTP,'boring|||http://www.youtube.com/watch?v=c1KNZ...,boring <link> mhm cap and carpenter s glasses...
1273,INFP,'The Americans It's a new show and I really ...,The Americans It s a new show and I really ...
3092,INTJ,No one under stands your sense of humor. 8653...,No one under stands your sense of humor. For ...
6376,ISTP,"'I admire you humor, INTP. So, it would be gre...","I admire you humor, INTP. So, it would be gre..."
7643,INFP,'I was wondering if anyone would take the time...,I was wondering if anyone would take the time...


### Classification

#### Naive Bayesian

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB


text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

text_clf = text_clf.fit(train['preprocessed_posts'], train['type'])

### Prediction

In [36]:
from preprocessing import format_text

test = pd.DataFrame(test)
test['preprocessed_posts'] = test['posts'].apply(format_text)
test.head()

,type,posts,preprocessed_posts
4587,ISFP,"'Dear ISFJ Mother, I wish you were less of a w...","Dear ISFJ Mother, I wish you were less of a w..."
2786,INFJ,"'To me, I think you guys may be over analyzing...","To me, I think you guys may be over analyzing..."
2813,ENFP,"'NIHM While NIHM has her INTJ husband, I've go...","NIHM While NIHM has her INTJ husband, I ve go..."
3705,INTP,'I want 5 kids: - an astro/nuclear/theoretical...,I want 5 kids: - an astro/nuclear/theoretical...
5957,ISFP,'I have the same thing as well. I've noticed t...,I have the same thing as well. I ve noticed t...


In [37]:
predicted = text_clf.predict(test['preprocessed_posts'])
np.mean(predicted == test['type'])
# default: 0.2072
# stop_words: 0.2069
# stop_words & preprocessed: 0.2069

0.2069164265129683

In [40]:
from sklearn.model_selection import cross_validate

np.random.seed(1)

scoring = {'acc': 'accuracy',
           'f1_micro': 'f1_micro'}

results = cross_validate(text_clf, train['preprocessed_posts'], train['type'], cv=5, 
                          scoring=scoring, n_jobs=-1)

In [41]:
# print(results)
print("Accuracy: {:0.4f} (+/- {:0.4f})".format(np.mean(results['test_acc']),
                                                          np.std(results['test_acc'])))

print("F1: {:0.4f} (+/- {:0.4f})".format(np.mean(results['test_f1_micro']),
                                                          np.std(results['test_f1_micro'])))


# Accuracy: 0.2140 (+/- 0.0004)
# F1: 0.2140 (+/- 0.0004)

# Preprocessed
# Accuracy: 0.2144 (+/- 0.0006)
# F1: 0.2144 (+/- 0.0006)

Accuracy: 0.2144 (+/- 0.0006)
F1: 0.2144 (+/- 0.0006)


#### SVM

In [34]:
from sklearn.linear_model import SGDClassifier

text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(max_iter=1000, tol=1e-3)),
                    ])

text_clf_svm = text_clf_svm.fit(X_train, y_train)

predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)
# default : 0.6585
# stop_words : 0.6571

0.6585014409221902